<a href="https://colab.research.google.com/github/afnan123-student/Training-Pistol-Datasets/blob/main/Partal_Guns_Detection_Individual_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Apr 15 22:33:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with TensorFlow Lite Model Maker

In this colab notebook, you'll learn how to use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) to train a custom object detection model to detect Android figurines and how to put the model on a Raspberry Pi.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.3/577.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

Import the required packages.

In [ ]:
pip install numpy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/MMU.zip

Archive:  /content/drive/MyDrive/MMU.zip
   creating: MMU/
  inflating: __MACOSX/._MMU          
   creating: MMU/validate/
  inflating: __MACOSX/MMU/._validate  
  inflating: MMU/.DS_Store           
  inflating: __MACOSX/MMU/._.DS_Store  
   creating: MMU/train/
  inflating: __MACOSX/MMU/._train    
  inflating: MMU/validate/istockphoto-179311497-612x612.xml  
  inflating: MMU/validate/575050366ea6baeb83eec0266752b28b.jpeg  
  inflating: __MACOSX/MMU/validate/._575050366ea6baeb83eec0266752b28b.jpeg  
  inflating: MMU/validate/man-hides-weapon-jeans-field-grass-setting-sun-hidden-gun-pocket-rear-view-214024697.xml  
  inflating: MMU/validate/12-Bra-Holster.jpeg  
  inflating: __MACOSX/MMU/validate/._12-Bra-Holster.jpeg  
  inflating: MMU/validate/ocf12012130-largejpg-dfccacff1cd4a0f0.jpeg  
  inflating: __MACOSX/MMU/validate/._ocf12012130-largejpg-dfccacff1cd4a0f0.jpeg  
  inflating: MMU/validate/download (1).xml  
  inflating: MMU/validate/635748780949579748-gun-DFP-feedback-0319.JP-

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'MMU/train',
    'MMU/train',
    ['Gun']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'MMU/validate',
    'MMU/validate',
    ['Gun']
)

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [ ]:
spec = model_spec.get('efficientdet_lite2')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 20`, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
75/75 [==============================] - 104s 684ms/step - det_loss: 1.3922 - cls_loss: 0.8671 - box_loss: 0.0105 - reg_l2_loss: 0.0759 - loss: 1.4681 - learning_rate: 0.0065 - gradient_norm: 5.3795 - val_det_loss: 1.0218 - val_cls_loss: 0.5870 - val_box_loss: 0.0087 - val_reg_l2_loss: 0.0760 - val_loss: 1.0978
Epoch 2/20
75/75 [==============================] - 46s 619ms/step - det_loss: 0.8957 - cls_loss: 0.5042 - box_loss: 0.0078 - reg_l2_loss: 0.0761 - loss: 0.9718 - learning_rate: 0.0049 - gradient_norm: 7.9081 - val_det_loss: 0.7531 - val_cls_loss: 0.4797 - val_box_loss: 0.0055 - val_reg_l2_loss: 0.0762 - val_loss: 0.8294
Epoch 3/20
75/75 [==============================] - 49s 653ms/step - det_loss: 0.7912 - cls_loss: 0.4445 - box_loss: 0.0069 - reg_l2_loss: 0.0764 - loss: 0.8676 - learning_rate: 0.0048 - gradient_norm: 8.1123 - val_det_loss: 0.8820 - val_cls_loss: 0.5579 - val_box_loss: 0.0065 - val_reg_l2_loss: 0.0765 - val_loss: 0.9585
Epoch 4/20
75/75 [============

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 10 images in the validation dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
model.evaluate(val_data)

5/5 [==============================] - 38s 3s/step



{'AP': 0.6818808,
 'AP50': 0.98501694,
 'AP75': 0.8374884,
 'APs': -1.0,
 'APm': 0.56141037,
 'APl': 0.7312289,
 'ARmax1': 0.6510386,
 'ARmax10': 0.7587537,
 'ARmax100': 0.76379824,
 'ARs': -1.0,
 'ARm': 0.6696078,
 'ARl': 0.8046808,
 'AP_/Gun': 0.6818808}

### Step 5: Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.

In [ ]:
model.export(export_dir='.', tflite_filename='HalfGun.tflite')